In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

In [4]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [5]:
# 初始化嵌入模型
embeddings = OpenAIEmbeddings()

<ipython-input-5-b0fd89d06ed4>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [6]:
# 创建商品列表（模拟文档）
products = [
    Document(page_content="这款智能手机拥有强大的处理器和高清摄像头。"),
    Document(page_content="这款笔记本电脑轻薄便携，适合商务旅行。"),
    Document(page_content="这款智能手表支持健康监测和消息提醒。")
]

In [9]:
# 创建向量存储
vectorstore = FAISS.from_documents(products, embeddings)

In [10]:
# 查询向量存储
query = "最好的智能手机"
results = vectorstore.similarity_search(query, k=2)

In [11]:
# 打印结果
for doc in results:
    print(doc.page_content)

这款智能手机拥有强大的处理器和高清摄像头。
这款智能手表支持健康监测和消息提醒。
